In [1]:
!pip install python-docx  
!pip install PyPDF2  


In [11]:
import subprocess

def check_ollama():
    try:
        result = subprocess.run(["ollama", "--version"], capture_output=True, text=True, check=True)
        print("Ollama is installed. Version:", result.stdout.strip())
    except FileNotFoundError:
        print("Ollama is NOT installed.")
    except subprocess.CalledProcessError as e:
        print("Error checking Ollama:", e)

check_ollama()


Ollama is installed. Version: ollama version is 0.6.3


In [22]:
import requests
from docx import Document
import PyPDF2
import subprocess

# Ollama API endpoint (Mac default)
OLLAMA_URL = "http://localhost:11434/v1/chat/completions"

# Function to check if Ollama is running
def check_ollama_running():
    try:
        response = requests.get("http://localhost:11434")
        if response.status_code == 200:
            print("✅ Ollama is running.")
            return True
    except requests.ConnectionError:
        print("❌ Ollama is NOT running. Start it by running 'ollama serve' in a terminal.")
        return False

# Function to send a prompt to Ollama
def ask_ollama(prompt):
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "llama3",  # Updated to LLaMA 3
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7  # Adjust for creativity
    }

    # Send request to Ollama
    response = requests.post(OLLAMA_URL, headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# Function to extract text from Word document (.docx)
def extract_text_from_docx(file_path):
    try:
        doc = Document(file_path)
        return '\n'.join([para.text for para in doc.paragraphs])
    except Exception as e:
        print(f"❌ Error reading DOCX file: {e}")
        return None

# Function to extract text from PDF document
def extract_text_from_pdf(file_path):
    try:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            return '\n'.join([page.extract_text() for page in reader.pages if page.extract_text()])
    except Exception as e:
        print(f"❌ Error reading PDF file: {e}")
        return None

# Function to analyze the CV with the job description using Ollama
def analyze_cv_with_job_description(cv_file_path, job_description_file_path):
    # Check if Ollama is running
    if not check_ollama_running():
        return "Ollama is not running. Please start it first."

    # Extract text from the resume and job description files
    if cv_file_path.endswith(".docx"):
        cv_text = extract_text_from_docx(cv_file_path)
    elif cv_file_path.endswith(".pdf"):
        cv_text = extract_text_from_pdf(cv_file_path)
    else:
        return "❌ Unsupported CV file format"

    if job_description_file_path.endswith(".docx"):
        job_description_text = extract_text_from_docx(job_description_file_path)
    elif job_description_file_path.endswith(".pdf"):
        job_description_text = extract_text_from_pdf(job_description_file_path)
    else:
        return "❌ Unsupported Job Description file format"

    # If extraction failed, return an error
    if not cv_text or not job_description_text:
        return "❌ Failed to extract text from one or both files."

    # Construct the prompt for Ollama
    prompt = f"""
    Compare the following resume with the job description:
    Resume: {cv_text}
    
    Job Description: {job_description_text}

    Provide a similarity score (0-100), highlight key matches, and suggest areas to improve in the resume.
    """

    # Get the analysis from Ollama
    result = ask_ollama(prompt)
    return result if result else "❌ Error analyzing resume."

# Example usage (Replace with actual file paths)
cv_file_path = "resume.docx"  # Replace with actual file path
job_description_file_path = "job_description.docx"  # Replace with actual file path

# Run the analysis
result = analyze_cv_with_job_description(cv_file_path, job_description_file_path)
print(result)


✅ Ollama is running.
Similarity Score: 85%

Key Matches:

* Both the resume and job description mention IT support, network administration, system maintenance, troubleshooting, and technical assistance.
* The resume highlights experience with Active Directory, Azure, anti-virus software, backup systems, and firewalls, which are all relevant to the job description's requirements for administering network and data security.
* The resume also emphasizes scripting skills using languages like PowerSheel, Visual Basic Script, and batch files, which aligns with the job description's request for experience in maintaining and installing IT systems.

Areas to Improve:

1. **Job-specific keywords**: While the resume mentions some relevant technical terms, it could include more specific keywords from the job description, such as "directory management," "group policy administration," "firewall configuration," or "data backup procedures."
2. **Quantifiable achievements**: The resume focuses on listi

In [19]:
#This one is by OLLAMA
import requests
from docx import Document
import PyPDF2

# Ollama API endpoint (adjust if needed)
OLLAMA_URL = "http://localhost:11434/v1/chat/completions"

# Function to send a prompt to Ollama
def ask_ollama(prompt):
    headers = {
        "Content-Type": "application/json"
    }

    data = {
        "model": "llama2",  # Use llama2 or the model you're working with
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7  # Adjust temperature for creativity if needed
    }

    # Send the request
    response = requests.post(OLLAMA_URL, headers=headers, json=data)
    
    # Check if the response is successful
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# Function to extract text from Word document (.docx)
def extract_text_from_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

# Function to extract text from PDF document
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
        return text

# Function to analyze the CV with the job description using Ollama
def analyze_cv_with_job_description(cv_file_path, job_description_file_path):
    # Extract text from the resume and job description files
    if cv_file_path.endswith(".docx"):
        cv_text = extract_text_from_docx(cv_file_path)
    elif cv_file_path.endswith(".pdf"):
        cv_text = extract_text_from_pdf(cv_file_path)
    else:
        return "Unsupported CV file format"

    if job_description_file_path.endswith(".docx"):
        job_description_text = extract_text_from_docx(job_description_file_path)
    elif job_description_file_path.endswith(".pdf"):
        job_description_text = extract_text_from_pdf(job_description_file_path)
    else:
        return "Unsupported Job Description file format"

    # Construct the prompt for Ollama
    prompt = f"""
    Compare the following resume with the job description:
    Resume: {cv_text}
    Job Description: {job_description_text}
    Provide a similarity score, and suggest areas to improve in the resume to make it better aligned with the job description.
    """

    # Get the analysis from Ollama
    result = ask_ollama(prompt)
    return result

# Example usage
cv_file_path = "resume.docx"  # Replace with your actual file path
job_description_file_path = "job_description.docx"  # Replace with your actual file path

# Get the analysis
result = analyze_cv_with_job_description(cv_file_path, job_description_file_path)
print(result)


Error: 404, {"error":{"message":"model \"llama2\" not found, try pulling it first","type":"api_error","param":null,"code":null}}

None


In [17]:
import subprocess

# Pull the llama3 model
subprocess.run(["ollama", "pull", "llama3"])


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 6a0746a1ec1a... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


CompletedProcess(args=['ollama', 'pull', 'llama3'], returncode=0)

import requests
from docx import Document
import PyPDF2
import subprocess

# Ollama API endpoint (Mac default)
OLLAMA_URL = "http://localhost:11434/v1/chat/completions"

# Function to check if Ollama is running
def check_ollama_running():
    try:
        response = requests.get("http://localhost:11434")
        if response.status_code == 200:
            print("✅ Ollama is running.")
            return True
    except requests.ConnectionError:
        print("❌ Ollama is NOT running. Start it by running 'ollama serve' in a terminal.")
        return False

# Function to send a prompt to Ollama
def ask_ollama(prompt):
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "llama3",  # Updated to LLaMA 3
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7  # Adjust for creativity
    }

    # Send request to Ollama
    response = requests.post(OLLAMA_URL, headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# Function to extract text from Word document (.docx)
def extract_text_from_docx(file_path):
    try:
        doc = Document(file_path)
        return '\n'.join([para.text for para in doc.paragraphs])
    except Exception as e:
        print(f"❌ Error reading DOCX file: {e}")
        return None

# Function to extract text from PDF document
def extract_text_from_pdf(file_path):
    try:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            return '\n'.join([page.extract_text() for page in reader.pages if page.extract_text()])
    except Exception as e:
        print(f"❌ Error reading PDF file: {e}")
        return None

# Function to analyze the CV with the job description using Ollama
def analyze_cv_with_job_description(cv_file_path, job_description_file_path):
    # Check if Ollama is running
    if not check_ollama_running():
        return "Ollama is not running. Please start it first."

    # Extract text from the resume and job description files
    if cv_file_path.endswith(".docx"):
        cv_text = extract_text_from_docx(cv_file_path)
    elif cv_file_path.endswith(".pdf"):
        cv_text = extract_text_from_pdf(cv_file_path)
    else:
        return "❌ Unsupported CV file format"

    if job_description_file_path.endswith(".docx"):
        job_description_text = extract_text_from_docx(job_description_file_path)
    elif job_description_file_path.endswith(".pdf"):
        job_description_text = extract_text_from_pdf(job_description_file_path)
    else:
        return "❌ Unsupported Job Description file format"

    # If extraction failed, return an error
    if not cv_text or not job_description_text:
        return "❌ Failed to extract text from one or both files."

    # Construct the prompt for Ollama
    prompt = f"""
    Compare the following resume with the job description:
    Resume: {cv_text}
    
    Job Description: {job_description_text}

    Provide a similarity score (0-100), highlight key matches, and suggest areas to improve in the resume.
    """

    # Get the analysis from Ollama
    result = ask_ollama(prompt)
    return result if result else "❌ Error analyzing resume."

# Example usage (Replace with actual file paths)
cv_file_path = "resume.docx"  # Replace with actual file path
job_description_file_path = "job_description.docx"  # Replace with actual file path

# Run the analysis
result = analyze_cv_with_job_description(cv_file_path, job_description_file_path)
print(result)


In [5]:
#this one is from openai
!pip install python-docx  # For Word documents
!pip install PyPDF2  # For PDFs (Alternatively use pdfplumber)
!pip install openai  # For OpenAI API


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 4.1 MB/s eta 0:00:00


In [9]:
!git status


On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .ipynb_checkpoints/New_Test-checkpoint.ipynb
	modified:   New_Test.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [11]:
!git add .


In [13]:
!git commit -m "made new changes here with test"


[main 98e31a0] made new changes here with test
 2 files changed, 130 insertions(+), 24 deletions(-)


In [17]:
!git push origin main --force  # Use --force to overwrite the previous commit


error: src refspec # does not match any
error: src refspec Use does not match any
error: src refspec to does not match any
error: src refspec overwrite does not match any
error: src refspec the does not match any
error: src refspec previous does not match any
error: src refspec commit does not match any
error: failed to push some refs to 'https://github.com/deadpool-kha/Python-Assignment'


In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import requests
from docx import Document
import PyPDF2

# Ollama API endpoint (Mac default)
OLLAMA_URL = "http://localhost:11434/v1/chat/completions"

# Function to check if Ollama is running
def check_ollama_running():
    try:
        response = requests.get("http://localhost:11434")
        if response.status_code == 200:
            print("✅ Ollama is running.")
            return True
    except requests.ConnectionError:
        print("❌ Ollama is NOT running. Start it by running 'ollama serve' in a terminal.")
        return False

# Function to send a prompt to Ollama
def ask_ollama(prompt):
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "llama3",  # Updated to LLaMA 3
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7  # Adjust for creativity
    }

    # Send request to Ollama
    response = requests.post(OLLAMA_URL, headers=headers, json=data)
    
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# Function to extract text from Word document (.docx)
def extract_text_from_docx(file_path):
    try:
        doc = Document(file_path)
        return '\n'.join([para.text for para in doc.paragraphs])
    except Exception as e:
        print(f"❌ Error reading DOCX file: {e}")
        return None

# Function to extract text from PDF document
def extract_text_from_pdf(file_path):
    try:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            return '\n'.join([page.extract_text() for page in reader.pages if page.extract_text()])
    except Exception as e:
        print(f"❌ Error reading PDF file: {e}")
        return None

# Function to analyze the CV with the job description using Ollama
def analyze_cv_with_job_description(cv_file_path, job_description_file_path):
    # Check if Ollama is running
    if not check_ollama_running():
        return "Ollama is not running. Please start it first."

    # Extract text from the resume and job description files
    if cv_file_path.endswith(".docx"):
        cv_text = extract_text_from_docx(cv_file_path)
    elif cv_file_path.endswith(".pdf"):
        cv_text = extract_text_from_pdf(cv_file_path)
    else:
        return "❌ Unsupported CV file format"

    if job_description_file_path.endswith(".docx"):
        job_description_text = extract_text_from_docx(job_description_file_path)
    elif job_description_file_path.endswith(".pdf"):
        job_description_text = extract_text_from_pdf(job_description_file_path)
    else:
        return "❌ Unsupported Job Description file format"

    # If extraction failed, return an error
    if not cv_text or not job_description_text:
        return "❌ Failed to extract text from one or both files."

    # Construct the prompt for Ollama
    prompt = f"""
    Compare the following resume with the job description:
    Resume: {cv_text}
    
    Job Description: {job_description_text}

    Provide a similarity score (0-100), highlight key matches, and suggest areas to improve in the resume.
    """

    # Get the analysis from Ollama
    result = ask_ollama(prompt)
    return result if result else "❌ Error analyzing resume."

# GUI for file selection and result display
def browse_cv():
    cv_file_path.set(filedialog.askopenfilename(title="Select CV file", filetypes=[("Word Files", "*.docx"), ("PDF Files", "*.pdf")]))
    
def browse_job_description():
    job_description_file_path.set(filedialog.askopenfilename(title="Select Job Description file", filetypes=[("Word Files", "*.docx"), ("PDF Files", "*.pdf")]))

def analyze():
    cv_path = cv_file_path.get()
    job_desc_path = job_description_file_path.get()
    
    if not cv_path or not job_desc_path:
        messagebox.showerror("Error", "Please select both CV and Job Description files.")
        return
    
    result = analyze_cv_with_job_description(cv_path, job_desc_path)
    
    result_text.delete(1.0, tk.END)
    result_text.insert(tk.END, result)

# Create the main window
root = tk.Tk()
root.title("Resume & Job Description Analyzer")

# Create input fields and buttons
cv_file_path = tk.StringVar()
job_description_file_path = tk.StringVar()

# CV input section
tk.Label(root, text="Select CV File").pack(pady=5)
cv_button = tk.Button(root, text="Browse CV", command=browse_cv)
cv_button.pack(pady=5)

# Job Description input section
tk.Label(root, text="Select Job Description File").pack(pady=5)
job_desc_button = tk.Button(root, text="Browse Job Description", command=browse_job_description)
job_desc_button.pack(pady=5)

# Analyze button
analyze_button = tk.Button(root, text="Analyze", command=analyze)
analyze_button.pack(pady=10)

# Output section
tk.Label(root, text="Analysis Result:").pack(pady=5)
result_text = tk.Text(root, height=15, width=100)
result_text.pack(pady=10)

# Start the Tkinter main loop
root.mainloop()


2025-03-30 21:27:35.562 python[18928:1714537] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-30 21:27:35.562 python[18928:1714537] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2025-03-30 21:28:25.057 python[18928:1714537] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'


✅ Ollama is running.


In [28]:
!git add .  
!git commit -m "Fixed secret leak"


[main 5f45103] Fixed secret leak
 2 files changed, 266 insertions(+), 252 deletions(-)


In [30]:
!git commit --amend --no-edit


[main 0b4e42e] Fixed secret leak
 Date: Sun Mar 30 21:52:11 2025 -0400
 2 files changed, 266 insertions(+), 252 deletions(-)


In [32]:
!git push origin main --force


remote: error: GH013: Repository rule violations found for refs/heads/main.        
remote: 
remote: - GITHUB PUSH PROTECTION        
remote:   â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”        
remote:     Resolve the following violations before pushing again        
remote: 
remote:     - Push cannot contain secrets        
remote: 
remote:             
remote:      (?) Learn how to resolve a blocked push        
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-command-line#resolving-a-blocked-push        
remote:             
remote:             
remote:       â€”â€” OpenAI API Key â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”        
remote:        locations:        
remote:          - commit: 4b8c1c7ea6f917af5b8cd305b7bc862c5323cbab    

In [36]:
!git add .  

In [38]:
!git commit -m "Added all files and folders to the repository"


[main 9408977] Added all files and folders to the repository
 1 file changed, 11 insertions(+), 18 deletions(-)


In [40]:
!git push origin main  # Push to the main branch of the remote repository


error: src refspec # does not match any
error: src refspec Push does not match any
error: src refspec to does not match any
error: src refspec the does not match any
error: src refspec branch does not match any
error: src refspec of does not match any
error: src refspec the does not match any
error: src refspec remote does not match any
error: src refspec repository does not match any
error: failed to push some refs to 'https://github.com/deadpool-kha/Python-Assignment'


In [42]:
!git branch


* main
  master


In [44]:
!git remote -v


origin	https://github.com/deadpool-kha/Python-Assignment (fetch)
origin	https://github.com/deadpool-kha/Python-Assignment (push)


In [46]:
!git log


commit 94089777254094d8e31a0e2411b4ef97c96133fe
Author: Angad Khanal <khanalak07@gmail.com>
Date:   Sun Mar 30 21:54:57 2025 -0400

    Added all files and folders to the repository

commit 0b4e42eea9b299427c48dbea77ae88992dc1e466
Author: Angad Khanal <khanalak07@gmail.com>
Date:   Sun Mar 30 21:52:11 2025 -0400

    Fixed secret leak

commit b9f6a354539f60b37e0e9269f8b3d5f2b796d248
Author: Angad Khanal <khanalak07@gmail.com>
Date:   Sun Mar 30 21:49:12 2025 -0400

    made new changes here with test

commit 4b8c1c7ea6f917af5b8cd305b7bc862c5323cbab
Author: Angad Khanal <khanalak07@gmail.com>
Date:   Sun Mar 30 21:45:12 2025 -0400

    made new test file which works

commit f56ff5e17e02abd2fc422103560581ec67d22fa7
Author: Angad Khanal <khanalak07@gmail.com>
Date:   Sat Mar 29 23:33:03 2025 -0400

    Added new notebook .ipynb

commit 1187006e9e3b7ad7c4b6dae66e27410f46bf8277
Author: Angad Khanal <khanalak07@gmail.com>
Date:   Sun Mar 23 00:18:35 2025 -0400

    U3

commit e850b891bfa1ba6

In [56]:
!git add . 
!git commit -m "Initial commit with files and folders"  


[main 25bf7af] Initial commit with files and folders
 1 file changed, 189 insertions(+), 1 deletion(-)


In [58]:
!git push origin main  


remote: error: GH013: Repository rule violations found for refs/heads/main.        
remote: 
remote: - GITHUB PUSH PROTECTION        
remote:   â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”        
remote:     Resolve the following violations before pushing again        
remote: 
remote:     - Push cannot contain secrets        
remote: 
remote:             
remote:      (?) Learn how to resolve a blocked push        
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-command-line#resolving-a-blocked-push        
remote:             
remote:             
remote:       â€”â€” OpenAI API Key â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”        
remote:        locations:        
remote:          - commit: 4b8c1c7ea6f917af5b8cd305b7bc862c5323cbab    